In [23]:
import numpy as np
import onnx
import onnxruntime as ort
import time
import os
import matplotlib.pyplot as plt
from scipy.signal import convolve2d
import csv

class Multiply_experiment_class():
    #TODO: Set the input size to the size of a layer in the TINYYOLO 
    def __init__(self):
        self.in1 = np.random.random([5,5]).astype(np.float32)
        self.in2 = np.random.random([28,28]).astype(np.float32)
        self.ort_runtime=[]
        self.out1 = np.random.random([24,24]).astype(np.float32)
        
    def expected_res_conv(self):
        res = convolve2d(self.in1,self.in2)
        return res

    def test_ort_mul(self, model_name):
        model_load = onnx.load(model_name)
        self.expected_result = self.expected_res_conv()
        input_dict = {"in1": self.in1, "in2": self.in2}
        sess = ort.InferenceSession(model_load.SerializeToString())
        self.out1 = sess.run(None,input_dict)           
    
    def throughput_test(self,interations):
        for i in range(interations):
            start = time.time()
            self.test_ort_mul("convolution_model.onnx")
            end = time.time()
            self.ort_runtime.append(end-start)       
        # return self.ort_runtime
    
    def results_publisher(self, filename = "ort_results.csv"):
        print("Interations = %d" %(len(self.ort_runtime)))
        print("Min  = %f ms" %(np.mean(self.ort_runtime)*1000))
        print("Max  = %f ms" %(np.max(self.ort_runtime)*1000))
        print("Mean = %f ms" %(np.min(self.ort_runtime)*1000))        
        if not os.path.exists("results/"):
            os.makedirs("results/")
        np.savetxt("results/" + filename , np.asarray(self.ort_runtime), delimiter=',')

In [24]:
t1 = Multiply_experiment_class()
t1.throughput_test(1)
if np.allclose(t1.out1,t1.expected_res_conv()):
    t1.results_publisher()
else:
    print("Outputs do not match!")

Fail: [ONNXRuntimeError] : 1 : FAIL : Node (Conv1) Op (Conv) [ShapeInferenceError] Attribute kernel_shape has incorrect size